In [45]:
from tensorflow_probability.substrates import jax as tfp
import jax
rng_key = jax.random.PRNGKey(10)
tfd = tfp.distributions
# logits = [[0,1],[1,0]]
logits = tfd.Normal(0,10).sample(seed=rng_key, sample_shape=(50,2))
outputs = [1,0]
log_lik = tfd.Bernoulli(logits=logits).log_prob(outputs)
print(log_lik.shape)
print(log_lik.sum())

(50, 2)
-476.06305


In [2]:
import torch

def sum_all_valus(a,b,c):
    print(a,b,c)
    return a+b+c

apply_data1 =  torch.vmap(sum_all_valus)
# loglikelihood of y
# print(n_of_positives.shape,group_size.shape,thetas.shape)
a = torch.tensor([1,2,3])
b = torch.tensor([0.1,0.2,0.3])
c = torch.tensor([0.01,0.02,0.03])
# logprob_y = torch.sum(
#     apply_data(a, b, c)
# )
print(apply_data1(a, b, c))
# print(logprob_y)

BatchedTensor(lvl=1, bdim=0, value=
    tensor([1, 2, 3])
) BatchedTensor(lvl=1, bdim=0, value=
    tensor([0.1000, 0.2000, 0.3000])
) BatchedTensor(lvl=1, bdim=0, value=
    tensor([0.0100, 0.0200, 0.0300])
)
tensor([1.1100, 2.2200, 3.3300])


In [8]:
n_of_positives = torch.tensor( [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 5, 2, 5, 3, 2, 7, 7, 3, 3, 2, 9, 10, 4, 4, 4, 4, 4, 4, 4, 10, 4, 4, 4, 5, 11, 12, 5, 5, 6, 5, 6, 6, 6, 6, 16, 15, 15, 9, 4, ], dtype=torch.float32)
group_size = torch.tensor( [ 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 18, 18, 17, 20, 20, 20, 20, 19, 19, 18, 18, 25, 24, 23, 20, 20, 20, 20, 20, 20, 10, 49, 19, 46, 27, 17, 49, 47, 20, 20, 13, 48, 50, 20, 20, 20, 20, 20, 20, 20, 48, 19, 19, 19, 22, 46, 49, 20, 20, 23, 19, 22, 20, 20, 20, 52, 46, 47, 24, 14, ], dtype=torch.float32)
logits_shape = torch.tensor([n_of_positives.shape])
logits = torch.distributions.uniform.Uniform(-2, 2).sample(logits_shape)
thetas = torch.nn.functional.sigmoid(logits)
print(n_of_positives.shape,group_size.shape,thetas.shape)
def binomial(y, N, theta):
    print(y,N,theta)
    # return y+N+theta
    return torch.distributions.binomial.Binomial(N, probs=theta, validate_args=False).log_prob(y)
    # return y+N+theta
print(torch.distributions.binomial.Binomial(group_size[0], probs=thetas[0]).log_prob((n_of_positives[0])))
print(binomial(n_of_positives[0],group_size[0],thetas[0]))
apply_data2 =  torch.vmap(binomial)

# apply_data =  torch.vmap(lambda y, N, theta: torch.distributions.binomial.Binomial(N, probs=theta).log_prob(y))
print(apply_data2(n_of_positives,group_size,thetas))



torch.Size([71]) torch.Size([71]) torch.Size([71])
tensor(-11.5149)
tensor(0.) tensor(20.) tensor(0.4377)
tensor(-11.5149)
BatchedTensor(lvl=1, bdim=0, value=
    tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
             1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,
             2.,  2.,  2.,  1.,  5.,  2.,  5.,  3.,  2.,  7.,  7.,  3.,  3.,  2.,
             9., 10.,  4.,  4.,  4.,  4.,  4.,  4.,  4., 10.,  4.,  4.,  4.,  5.,
            11., 12.,  5.,  5.,  6.,  5.,  6.,  6.,  6.,  6., 16., 15., 15.,  9.,
             4.])
) BatchedTensor(lvl=1, bdim=0, value=
    tensor([20., 20., 20., 20., 20., 20., 20., 19., 19., 19., 19., 18., 18., 17.,
            20., 20., 20., 20., 19., 19., 18., 18., 25., 24., 23., 20., 20., 20.,
            20., 20., 20., 10., 49., 19., 46., 27., 17., 49., 47., 20., 20., 13.,
            48., 50., 20., 20., 20., 20., 20., 20., 20., 48., 19., 19., 19., 22.,
            46., 49., 20., 20., 23., 19., 22., 

In [9]:
import jax.numpy as jnp

# create transition matrix
prob_vect = jnp.repeat(0.5, 5)
# print(prob_vect)
transition_mat = jnp.diag(prob_vect, -1) + jnp.diag(prob_vect, 1)
transition_mat = transition_mat.at[0, 0].set(0.5)
transition_mat = transition_mat.at[5, 5].set(0.5)

def calculate_q_at_state_no(state_no, p_initial, transition_mat):
    transition_mat_at_state_no = jnp.linalg.matrix_power(transition_mat, state_no)
    print(transition_mat_at_state_no)
    q = jnp.matmul(transition_mat_at_state_no, p_initial)  # q = p_initial * T^n = transpose(T)^n * p_initial
    return q

x_0 = 2  # initial state
p_initial = [0] * 6
p_initial[x_0] = 1
p_initial = jnp.array(p_initial)
p_initial
states = [0, 1, 2, 3, 10, 100, 200, 400]
for ind, state_no in enumerate(states):
    print(transition_mat)
    q = calculate_q_at_state_no(
        state_no, p_initial, transition_mat
    )  # q is proposed probabilities for p(x) on particular markov state

[[0.5 0.5 0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.5 0.5]]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[0.5 0.5 0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.5 0.5]]
[[0.5 0.5 0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.5 0.5]]
[[0.5 0.5 0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.5 0.5]]
[[0.5  0.25 0.25 0.   0.   0.  ]
 [0.25 0.5  0.   0.25 0.   0.  ]
 [0.25 0.   0.5  0.   0.25 0.  ]
 [0.   0.25 0.   0.5  0.   0.25]
 [0.   0.   0.25 0.   0.5  0.25]
 [0.   0.   0.   0.25 0.25 0.5 ]]
[[0.5 0.5 0.  0.  0.  

In [6]:
import jax

@jax.jit
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jax.numpy.where(x > 0, x, alpha * jax.numpy.exp(x) - alpha)

key = jax.random.key(0)
x = jax.random.normal(key, (10,))
print(selu(x))

[-0.5448514   0.2774427  -0.29255113 -0.91421574 -0.62452507 -0.24748124
 -0.85743254 -0.7823266   0.7682733   0.59566766]


In [9]:
import jax
from jax import jit,lax
import tensorflow_probability.substrates.jax as tfp

rng_key = jax.random.PRNGKey(82)
tfd = tfp.distributions
n_rat_tumors = 71

jacobian_fn = jax.jacfwd(jax.nn.sigmoid)
jacobian_fn_softplus = jax.jacfwd(jax.nn.softplus)

def joint_log_prob(params):
    a, b, logits = params["a"], params["b"], params["logits"]

    # HMC requires unconstrained sample space, thus it samples logits and we apply change of variable to
    # get theta from logits
    thetas = jax.nn.sigmoid(logits)
    a = jax.nn.softplus(a)
    b = jax.nn.softplus(b)
    log_det_jacob = jnp.sum(
        jax.vmap(lambda logit: jnp.log(jnp.abs(jnp.linalg.det(jacobian_fn(logit.reshape(1, 1))))))(logits)
    )

    log_det_jacob_priors = jnp.log(jnp.abs(jacobian_fn(a))) + jnp.log(jnp.abs(jacobian_fn(b)))

    # improper prior for a,b
    logprob_ab = jnp.log(jnp.power(a + b, -2.5))

    # logprob prior of theta
    logprob_thetas = tfd.Beta(a, b).log_prob(thetas).sum()

    # loglikelihood of y
    logprob_y = jnp.sum(
        jax.vmap(lambda y, N, theta: tfd.Binomial(N, probs=theta).log_prob(y))(n_of_positives, group_size, thetas)
    )
    print('joint_log_prob')
    return logprob_ab + logprob_thetas + logprob_y + log_det_jacob + log_det_jacob_priors

    # return logprob_ab + logprob_thetas + logprob_y + log_det_jacob

def init_param_fn(seed):
    """
    initialize a, b & logits
    """
    key1, key2 = jax.random.split(seed)
    return {
        "a": tfd.Uniform(0, 3).sample(seed=key1),
        "b": tfd.Uniform(0, 3).sample(seed=key2),
        "logits": tfd.Uniform(-2, 2).sample(n_rat_tumors, seed),
    }

In [8]:

def kernel(key, state):
    output = [jax.numpy.where(state > 0, 1,-1),key]
    if output == 1:
        return output, 'greater than 1'
    else:
        return output, 'less than 1'

n_chains = 4
keys = jax.random.split(rng_key, n_chains)
initial_states = jax.vmap(lambda seed: kernel(init_param_fn(seed)))(keys)



def abc(num_samples,num_chains):
    @jit
    def xyz(states):
        keys = jax.random.split(rng_key, num_chains)
        states, infos = jax.vmap(kernel)(keys,states)
        return states, (states,infos)
    
    keys = jax.random.split(rng_key, num_samples)
    _, (states, infos) = lax.scan(xyz, 2.4, keys)


In [22]:
import tensorflow_probability.substrates.jax as tfp
import jax
key = jax.random.PRNGKey(82)
tfd = tfp.distributions

def uniform_data(seed):
    return {'a':tfd.Uniform(0, 3).sample(seed=seed)}

def states(position,seed):
    return (position,seed)

keys = jax.random.split(key,4)
initial_states = jax.vmap(lambda seed: states(uniform_data(seed),seed))(keys)
print(initial_states)

({'a': Array([2.8442535 , 2.4914398 , 1.910634  , 0.39569843], dtype=float32)}, Array([[4194276336, 3483324415],
       [ 652449214,  703397549],
       [1471379998, 3506494469],
       [2265643003,  811380549]], dtype=uint32))


In [11]:
import torch

# torch.dot(torch.tensor([1,2,3]),torch.tensor([1,2,3]))
# batch_dot = torch.func.vmap(torch.dot)
# batch_dot(torch.tensor([[1,2,3]]),torch.tensor([[1,2,3]]))
# x,y = torch.randn(2,5), torch.randn(2,5)
# print(x)
# print(y)
# print(batch_dot(x,y))

# f = lambda dict: torch.dot(dict['x'], dict['y'])
# input = [{'x': torch.randn(2, 5), 'y': torch.randn(5)} for _ in range(4)]
# batched_dot = torch.vmap(f, in_dims=0)
# batched_dot(input)

abc = torch.Tensor([[1,2,3,4],[5,6,7,8]])
print(abc.dim())
print(abc.shape[:1][0])

2
2


In [18]:
from jax.flatten_util import ravel_pytree
import tensorflow_probability.substrates.jax as tfp
import jax

tfd = tfp.distributions
rng_key = jax.random.PRNGKey(82)
key1, key2 = jax.random.split(rng_key)
position =  {
        'a'  : tfd.Uniform(0, 3, validate_args=False).sample(seed=key1),
        'b'  : tfd.Uniform(0, 3, validate_args=False).sample(seed=key2),
        'logits' :  tfd.Uniform(-2, 2, validate_args=False).sample(sample_shape=(71,),seed=rng_key)}
p, unravel_fn = ravel_pytree(position)
print(position)
print(p)
print(unravel_fn)
print(p.shape)


{'a': Array(2.4621992, dtype=float32), 'b': Array(1.5081836, dtype=float32), 'logits': Array([-0.5830965 ,  0.9119353 , -1.0459356 ,  0.6607442 ,  0.05115604,
        1.9180942 , -0.8123336 , -0.44906664, -0.7647896 ,  1.3639178 ,
       -1.2365346 , -0.5110321 , -0.36780643, -0.30413723,  1.486865  ,
       -1.1990237 , -0.10980558,  1.2190585 ,  0.11574888, -0.76580906,
       -1.3776202 ,  1.7598639 ,  0.20172453,  0.601172  ,  1.3165002 ,
       -0.50377035,  0.03355122,  0.59856844,  0.6224303 , -0.5276737 ,
        1.7546191 , -0.20318699, -1.401957  ,  0.3111601 ,  1.4075761 ,
       -0.18194914,  1.5353355 , -1.3714151 , -0.9289584 ,  1.684566  ,
        0.27385235, -0.04773569, -1.4597545 ,  1.8086128 , -0.39681387,
       -1.822547  , -0.00653982,  1.7390656 ,  1.6817617 ,  1.7226138 ,
       -0.88208246,  1.4745278 , -0.20355034,  1.624649  , -0.10385036,
       -0.49547052, -1.1517582 , -0.5216894 ,  0.3182559 ,  1.5142751 ,
       -0.41060638,  0.3635683 , -0.2908492 ,  1.

In [17]:
import jax
import jax.numpy as jnp
from jax.random import normal
from jax.flatten_util import ravel_pytree
from functools import partial
from jax import jit,lax
import tensorflow_probability.substrates.jax as tfp

tfd = tfp.distributions
n_rat_tumors = 71
# inverse_mass_matrix = jnp.array([0.5] * (73))
# print(inverse_mass_matrix)
# print(jnp.sqrt(jnp.reciprocal(inverse_mass_matrix)))
@partial(jit, static_argnames=("precision",), inline=True)
def linear_map(diag_or_dense_a, b, *, precision="highest"):
    dtype = jnp.result_type(diag_or_dense_a.dtype, b.dtype)
    diag_or_dense_a = diag_or_dense_a.astype(dtype)
    b = b.astype(dtype)
    ndim = jnp.ndim(diag_or_dense_a)

    if ndim <= 1:
        return lax.mul(diag_or_dense_a, b)
    else:
        return lax.dot(diag_or_dense_a, b, precision=precision)
    
def init_param_fn(seed):
    """
    initialize a, b & logits
    """
    key1, key2 = jax.random.split(seed)
    return {
        "a": tfd.Uniform(0, 3).sample(seed=key1),
        "b": tfd.Uniform(0, 3).sample(seed=key2),
        "logits": tfd.Uniform(-2, 2).sample(n_rat_tumors, seed),
    }

rng_key = jax.random.PRNGKey(82)
position = init_param_fn(rng_key)
inverse_mass_matrix = jnp.array([0.5] * (73))
sigma = jnp.sqrt(jnp.reciprocal(inverse_mass_matrix))
p, unravel_fn = ravel_pytree(position)
sample = normal(rng_key, shape=p.shape, dtype=p.dtype)
print(unravel_fn(0.0 + linear_map(sigma, sample)))

{'a': Array(-0.07475591, dtype=float32), 'b': Array(-0.08839476, dtype=float32), 'logits': Array([-0.04861826, -1.8656483 ,  0.34041414,  0.18445395,  0.5349512 ,
        0.95408654, -3.0757098 , -1.1467196 ,  1.19135   , -0.2820422 ,
       -1.685374  ,  1.500075  , -0.13415635,  0.14802201,  1.5196137 ,
       -0.00603148, -0.7331637 , -1.5049448 , -0.73264414, -2.600379  ,
        1.4972322 ,  0.01504064,  1.7474204 , -2.1307764 , -1.1750268 ,
       -2.8646846 , -2.4126124 ,  0.9970587 ,  1.4150826 , -1.564853  ,
        0.9212672 ,  1.4154911 , -0.6433712 , -0.2716634 , -1.8312318 ,
       -1.8919958 , -1.7231661 , -0.6976004 , -0.23184645,  0.37453178,
        2.238924  ,  2.4347796 , -4.047677  ,  0.7205491 , -4.6773086 ,
        0.37422583, -0.48042172, -0.38190788, -1.366039  , -0.40215185,
       -0.14671317, -0.3983269 , -0.50269324,  0.30281428,  2.2282526 ,
       -2.3805358 , -1.4691638 ,  0.09317751, -1.2908531 ,  1.7081944 ,
       -1.8053311 , -2.2225502 ,  1.6838262 ,

In [11]:
import torch
# print(f'means : {torch.zeros(71)}')
# print(f'covariance : {torch.eye(71)}')
dist = torch.distributions.normal.Normal(loc=torch.zeros(71),scale=0.5*torch.ones(71))
print(dist.sample())

tensor([-0.5718,  0.2643,  1.0009,  0.0569, -0.5621, -0.6192, -0.5267,  0.2028,
        -0.9483, -0.1262,  1.1488, -0.5866,  0.4012,  0.2000,  0.4527,  0.7122,
        -0.0744, -0.6050, -0.1402, -0.0629, -0.0578,  0.2841, -0.0971, -0.6210,
         0.3933, -0.6801, -0.1018, -0.0279,  0.2880,  0.1497, -1.1574, -0.1302,
        -0.3572,  0.4622,  0.0133,  0.3708,  0.0758, -1.0541, -0.4030,  0.0652,
         0.2437,  0.3255,  0.3924, -1.2665, -0.5735, -0.0120,  0.2569, -0.2013,
        -0.1668,  0.9001,  0.7111, -0.0732, -0.7182,  0.1098,  0.7383,  0.2254,
        -0.4604,  0.5400, -0.4769, -0.0024, -0.1629,  0.2353,  0.0550,  0.7600,
        -0.3215,  0.1037,  0.3392, -0.1784, -0.6881,  0.5868,  0.4467])


In [52]:
# theta
from typing import NamedTuple
import torch

n_of_positives = torch.tensor( [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 5, 2, 5, 3, 2, 7, 7, 3, 3, 2, 9, 10, 4, 4, 4, 4, 4, 4, 4, 10, 4, 4, 4, 5, 11, 12, 5, 5, 6, 5, 6, 6, 6, 6, 16, 15, 15, 9, 4, ], dtype=torch.float32)
group_size = torch.tensor( [ 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 18, 18, 17, 20, 20, 20, 20, 19, 19, 18, 18, 25, 24, 23, 20, 20, 20, 20, 20, 20, 10, 49, 19, 46, 27, 17, 49, 47, 20, 20, 13, 48, 50, 20, 20, 20, 20, 20, 20, 20, 48, 19, 19, 19, 22, 46, 49, 20, 20, 23, 19, 22, 20, 20, 20, 52, 46, 47, 24, 14, ], dtype=torch.float32)

jacobian_fn = torch.func.jacfwd(torch.nn.functional.sigmoid)
jacobian_fn_softplus = torch.func.jacfwd(torch.nn.functional.softplus)

class Position(NamedTuple):
    a: torch.Tensor
    b: torch.Tensor
    logits: torch.Tensor

def joint_log_prob(params: Position) -> torch.Tensor:
    a, b, logits = params.a, params.b, params.logits

    thetas = torch.nn.functional.sigmoid(logits)
    a = torch.nn.functional.softplus(a)
    b = torch.nn.functional.softplus(b)

    log_det_jacob = torch.sum(
        torch.vmap(lambda logit: torch.log(torch.abs(torch.linalg.det(jacobian_fn(logit.reshape(1, 1))))))(logits)
    )

    # improper prior for a,b
    logprob_ab = torch.log(torch.float_power(a + b, -2.5))

    # logprob prior of theta
    logprob_thetas = torch.distributions.beta.Beta(a, b).log_prob(thetas).sum()

    apply_data =  torch.vmap(lambda y, N, theta: torch.distributions.binomial.Binomial(N, probs=theta, validate_args=False).log_prob(y))
    # loglikelihood of y
    logprob_y = torch.sum(
       apply_data(n_of_positives, group_size, thetas)
    )
    return logprob_ab + logprob_thetas + logprob_y + log_det_jacob


# def update_fn(current_tensor:torch.Tensor):
#     return current_tensor*torch.distributions.uniform.Uniform(0, 3, validate_args=False).sample(sample_shape=(current_tensor.shape[-1],))

pos1 = Position(
        a = torch.distributions.uniform.Uniform(0, 3, validate_args=False).sample(),
        b = torch.distributions.uniform.Uniform(0, 3, validate_args=False).sample(),
        logits =  torch.distributions.uniform.Uniform(-2, 2, validate_args=False).sample(sample_shape=(71,)),
    )
pos2 = Position(
        a = torch.distributions.uniform.Uniform(0, 3, validate_args=False).sample(),
        b = torch.distributions.uniform.Uniform(0, 3, validate_args=False).sample(),
        logits =  torch.distributions.uniform.Uniform(-2, 2, validate_args=False).sample(sample_shape=(71,)),
    )
print(joint_log_prob(pos1))
print(joint_log_prob(pos2))
print(torch.subtract(joint_log_prob(pos2),joint_log_prob(pos1)))
print(torch.gradient(torch.tensor([joint_log_prob(pos1),joint_log_prob(pos2)])))
# # print(len(pos.a.shape))
# print(pos.a.unsqueeze(0).shape[-1])
# print(pos.logits.unsqueeze(0).shape[-1])
# pos_update_dict = {}
# for name,value in zip(pos._fields,pos):
#     pos_update_dict[name] = update_fn(value)

# new_pos = Position(**pos_update_dict)
# print(new_pos)
# torch.gradient(torch.tensor([1,2]))

tensor(-1056.9024, dtype=torch.float64)
tensor(-956.7878, dtype=torch.float64)
tensor(100.1146, dtype=torch.float64)
(tensor([100.1146, 100.1146], dtype=torch.float64),)


In [5]:
import torch
# from torch.func import grad

# x = torch.randn([])
# cos_x = grad(lambda x: torch.sin(x))(x)
# assert torch.allclose(cos_x, x.cos())
# print(x,x.cos(),cos_x)
# torch.distributions.uniform.Uniform(0, 3, validate_args=False).sample(sample_shape=(4,))
sample = torch.distributions.uniform.Uniform(-2, 2, validate_args=False).sample(sample_shape=(71,4))
print(sample)
                                                                                
                                                                        


tensor([[ 3.9589e-01, -1.8276e+00,  1.3651e+00, -1.9453e+00],
        [-6.9609e-01, -1.9410e+00, -1.8210e+00, -1.9413e+00],
        [-6.2078e-01, -6.7969e-01, -5.5754e-01,  1.3908e+00],
        [ 1.9284e+00, -3.4854e-01, -3.5880e-01, -1.4026e+00],
        [ 1.5286e+00, -5.2742e-01,  8.7124e-01,  1.0254e+00],
        [-8.8449e-01, -1.3673e+00, -3.7122e-01, -8.2427e-01],
        [ 1.1585e+00,  3.3126e-02,  1.4649e+00, -8.7469e-01],
        [-1.5747e+00, -9.2794e-01,  2.5389e-01,  1.0795e+00],
        [-7.1296e-01,  1.5220e+00,  2.6638e-01,  1.3141e+00],
        [ 6.8450e-01, -1.1606e+00, -3.6868e-01, -1.9550e+00],
        [ 1.5599e+00,  1.2254e+00, -3.9266e-01, -1.8518e+00],
        [-1.6191e+00,  1.0354e+00,  6.1860e-01,  1.0131e+00],
        [-7.7582e-01, -3.2102e-01,  1.0088e+00,  1.5622e+00],
        [-1.8697e+00, -6.8815e-01, -1.2677e+00, -1.9908e+00],
        [-1.2291e+00, -1.9938e+00, -1.8720e+00, -1.7855e+00],
        [-1.8059e+00, -1.8125e+00,  6.5918e-01,  1.2546e+00],
        

In [3]:
import jax
import jax.numpy as jnp
import tensorflow_probability.substrates.jax as tfp

n_of_positives = jnp.array( [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 5, 2, 5, 3, 2, 7, 7, 3, 3, 2, 9, 10, 4, 4, 4, 4, 4, 4, 4, 10, 4, 4, 4, 5, 11, 12, 5, 5, 6, 5, 6, 6, 6, 6, 16, 15, 15, 9, 4, ], dtype=jnp.float32)
group_size = jnp.array( [ 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 18, 18, 17, 20, 20, 20, 20, 19, 19, 18, 18, 25, 24, 23, 20, 20, 20, 20, 20, 20, 10, 49, 19, 46, 27, 17, 49, 47, 20, 20, 13, 48, 50, 20, 20, 20, 20, 20, 20, 20, 48, 19, 19, 19, 22, 46, 49, 20, 20, 23, 19, 22, 20, 20, 20, 52, 46, 47, 24, 14, ], dtype=jnp.float32)
n_rat_tumors = len(group_size)  # number of different kind of rat tumors


tfd = tfp.distributions
jacobian_fn = jax.jacfwd(jax.nn.sigmoid)
jacobian_fn_softplus = jax.jacfwd(jax.nn.softplus)

def joint_log_prob(params):
    a, b, logits = params["a"], params["b"], params["logits"]

    # HMC requires unconstrained sample space, thus it samples logits and we apply change of variable to
    # get theta from logits
    thetas = jax.nn.sigmoid(logits)
    a = jax.nn.softplus(a)
    b = jax.nn.softplus(b)
    print(logits.shape)
    log_det_jacob = jnp.sum(
        jax.vmap(lambda logit: jnp.log(jnp.abs(jnp.linalg.det(jacobian_fn(logit.reshape(1, 1))))))(logits)
    )
    print(log_det_jacob.shape)

    log_det_jacob_priors = jnp.log(jnp.abs(jacobian_fn(a))) + jnp.log(jnp.abs(jacobian_fn(b)))

    # improper prior for a,b
    logprob_ab = jnp.log(jnp.power(a + b, -2.5))

    # logprob prior of theta
    logprob_thetas = tfd.Beta(a, b).log_prob(thetas).sum()

    # loglikelihood of y
    logprob_y = jnp.sum(
        jax.vmap(lambda y, N, theta: tfd.Binomial(N, probs=theta).log_prob(y))(n_of_positives, group_size, thetas)
    )
    return logprob_ab + logprob_thetas + logprob_y + log_det_jacob + log_det_jacob_priors

    # return logprob_ab + logprob_thetas + logprob_y + log_det_jacob

rng_key = jax.random.PRNGKey(82)

def init_param_fn(seed):
    """
    initialize a, b & logits
    """
    key1, key2 = jax.random.split(seed)
    return {
        "a": tfd.Uniform(0, 3).sample(seed=key1),
        "b": tfd.Uniform(0, 3).sample(seed=key2),
        "logits": tfd.Uniform(-2, 2).sample(71, seed),
    }

def init(val):
    return val

keys = jax.random.split(rng_key, 4)
initial_states = jax.vmap(lambda seed: init(init_param_fn(seed)))(keys)
print(joint_log_prob(initial_states))

(4, 71)


TypeError: reshape total size must be unchanged, got new_sizes (4, 1, 1) for shape (4, 71).

In [12]:
import torch

accpt = torch.min(torch.exp(torch.tensor([-1.5094884365301])), torch.ones(1))
print(accpt)
accept_condition = torch.distributions.Bernoulli(accpt).sample()
print(accept_condition.bool())
if accept_condition.bool():
    print(True)
else:
    print(False)

tensor([0.2210])
tensor([True])
True


In [ ]:
from jax import lax
lax.fori_loop(0, num_integrations_steps, one_step, initial_state)

In [48]:
import torch

print(0.5*torch.eye(3))
covariance = 0.5*torch.eye(3)
velocity = torch.distributions.MultivariateNormal(torch.zeros(covariance.shape[-1]),covariance).sample()
print(covariance.shape)
# velocity = velocity.unsqueeze(0)
print(velocity.shape)
print(0.5*torch.matmul(velocity, torch.matmul(covariance, velocity.T)))
# 0.5*torch.dot(velocity,torch.multiply(covariance, velocity))
# print(torch.matmul(torch.tensor([1,2,3]), torch.tensor([[1],[1],[1]])))

print(torch.where(accept_condition.bool(), HMCState(proposal_energy_parameters,None), torch.ten))

tensor([[0.5000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.5000]])
torch.Size([3, 3])
torch.Size([3])
tensor(0.5103)


In [9]:
import jax
import jax.numpy as jnp
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
import seaborn as sns
import os
import blackjax
from jax import jit, lax


# from probml_utils.blackjax_utils import arviz_trace_from_states, inference_loop_multiple_chains
from sklearn.datasets import load_iris

try:
    from tensorflow_probability.substrates import jax as tfp
except ModuleNotFoundError:
    %pip install -qqq tensorflow_probability
    from tensorflow_probability.substrates import jax as tfp

import arviz as az

tfd = tfp.distributions

In [26]:
iris = load_iris()
X = iris.data
y = iris.target

# Convert to pandas dataframe
df_iris = pd.DataFrame(data=iris.data, columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])
df_iris["species"] = pd.Series(iris.target_names[y], dtype="category")


df = df_iris.query("species == ('setosa', 'versicolor')")

df_balanced = df[25:75]
df_balanced.head()
X = df_balanced[["sepal_length", "sepal_width"]].values  # features
# X.shape
y = pd.Categorical(df_balanced["species"]).codes

bias_prior = tfd.Normal(0, 20)
weights_prior = tfd.Normal(0, 2)

def joint_logprob_fn(params, inputs=X, outputs=y):
    weights = params["weights"]
    bias = params["bias"]
    # prior logprob
    log_prior = weights_prior.log_prob(weights).sum() + bias_prior.log_prob(bias)
    # likelihood logprob
    logits = inputs @ weights + bias
    print(logits.shape,outputs.shape)
    log_lik = tfd.Bernoulli(logits=logits).log_prob(outputs).sum()
    print(log_lik)
    return log_prior + log_lik

# initialize the params for two chains
n_chains = 2
rng_key = jax.random.PRNGKey(10)
initial_params = {
    "weights": weights_prior.sample(seed=rng_key, sample_shape=(n_chains, 2)),
    "bias": bias_prior.sample(seed=rng_key, sample_shape=(n_chains,)),
}
print(joint_logprob_fn(initial_params))


# inverse_mass_matrix = jnp.array([0.5] * 3)
# step_size = 0.01
# nuts = blackjax.nuts(joint_logprob_fn, step_size, inverse_mass_matrix)

# initial_states = jax.vmap(nuts.init)(initial_params)
# kernel = jax.jit(nuts.step)

# def inference_loop_multiple_chains(rng_key, kernel, initial_states, num_samples, num_chains):
#     @jax.jit
#     def one_step(states, rng_key):
#         keys = jax.random.split(rng_key, num_chains)
#         states, infos = jax.vmap(kernel)(keys, states)
#         return states, (states, infos)
    
#     keys = jax.random.split(rng_key, num_samples)
#     _, (states, infos) = lax.scan(one_step, initial_states, keys)

#     return (states, infos)

(50, 2) (50,)


ValueError: Incompatible shapes for broadcasting: shapes=[(50, 2), (50,)]

In [13]:
%%time
rng_key = jax.random.PRNGKey(1)
states, infos = inference_loop_multiple_chains(rng_key, kernel, initial_states, 2000, n_chains)

TypeError: clip() got an unexpected keyword argument 'max'